
# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

----------------------------------------------------------------------------------------------------------------------------------------------------

## LIBRAIRIES

---

In [1]:
#import librairies

#API AND SCRAPPING
import requests
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import json
import re
import logging

#DATA MANIPULATION
import pandas as pd
import numpy as np
import time
import datetime

#VISUALISATION
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import folium
from folium import plugins

#STORAGE
from dotenv import load_dotenv
import os
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError


---

## OBTENTION DES COORDONNES GPS

---

Première étape, obtenir les coodonnées GPS des villes qui nous intéresse (via nominatim.org). Cela nous permettra de les places sur notre carte finale mais aussi d'obtenir la météo locale.
Concernant la liste, certains endroits ne sont pas des villes à proprement parler, mais nous verrons !

In [46]:
# Liste des villes à visiter 
cities_list= [
"Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg", # lieu touristique
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon", #zone touristique
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege", # département 
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"
]

Créeons une fonction pour récupérer les coordonnées GPS pour la liste ci-dessus (avec un petit délais pour ne pas surcharger l'API) et les stocker dans un Dataframe

In [ ]:
# import des coordonnées GPS de la liste des villes
def get_coordinates(cities_list: list[str], delay: float = 0.5) -> pd.DataFrame:
    """
    Récupère les coordonnées GPS pour une liste de villes.
    
    Args:
        cities_list (list[str]): Liste des noms de villes.
        delay (float): Temps d'attente entre deux requêtes (en secondes).
    
    Returns:
        pd.DataFrame: DataFrame contenant les coordonnées GPS.
    """
    url = "https://nominatim.openstreetmap.org/search"
    headers = {"User-Agent": "UniversityProject/1.0 (contact@example.com)"}
    results = []

    for city in cities_list:
        try:
            response = requests.get(
                url,
                params={"city": city, "format": "json"},
                headers=headers
            )
            response.raise_for_status()
            data = response.json()

            if data:
                results.append({
                    "City": city,
                    "Latitude": data[0].get("lat"),
                    "Longitude": data[0].get("lon")
                })
            else:
                print(f"Aucune donnée trouvée pour {city}")
        except requests.RequestException as e:
            print(f"Erreur pour {city}: {e}")

        time.sleep(delay)  # Respecter les limites de l'API

    return pd.DataFrame(results)


coordinates_df = get_coordinates(cities_list)

print(coordinates_df)


Les coordonnées du Chateau du Haut-Koenigsbourg et des Gorges du Verdon semblent cohérentes (elle le sont après vérifiction)..par contre la longitude -55.xx sur l'ariège pas du tout. Vérifions sur une carte


In [ ]:
# Coordonnées GPS de la France
latitude = 46.603354
longitude = 1.888334

# Création de la carte
map_france = folium.Map(location=[latitude, longitude], zoom_start=6)

# Ajout du marqueur pour la ville "ariège" (en rouge)
for _, row in coordinates_df.iterrows():
    if row["City"] == "Ariege":
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=row["City"],
            icon=folium.Icon(color="red")
        ).add_to(map_france)

# Affichage de la carte
map_france.save("map_france.html") # il faudra effectivement dézoomer pour voir le marque rouge
map_france


Remplaçons "Ariege" par la préfécture du département : "Foix"

In [ ]:
# Supprimer "Ariège"
cities_list.remove("Ariege")
# .. et ajouter "Foix"
cities_list.append("Foix")

In [ ]:
# coordonnées définitive des villes
coordinates_df = get_coordinates(cities_list)
print(coordinates_df)
coordinates_df.to_csv("src/coordonnees_villes.csv", index=False)

---

# OBTENTION DES INFOS METEOS PAR VILLE

---

## Récupération des données météorologiques via OpenWeatherMap API

L'API gratuite d'OpenWeatherMap utilisée dans ce projet présente certaines limitations :
- Fournit uniquement des prévisions (pas d'historique)
- Données par tranches de 3 heures (00:00, 03:00, 06:00, 09:00, 12:00, 15:00, 18:00, 21:00.)
- Prévisions sur 5 jours

Pour assurer la cohérence des données :
- Sélection d'un créneau horaire fixe (16h) pour toutes les villes
- Exclusion des températures min/max qui ne sont pertinentes que sur 3h
- Focus sur : température moyenne, humidité, vitesse du vent et précipitations

Note : Les précipitations indiquées correspondent aux prévisions sur 3h autour du créneau choisi.

In [ ]:
# Charger les variables d'environnement
load_dotenv('.secrets')
API_KEY = os.getenv("OPENWEATHER_API_KEY")

# Charger le DataFrame
df = pd.read_csv('src/coordonnees_villes.csv')

# Fonction pour récupérer les données météo

def get_weather(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units=metric&appid={API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        weather_data = []
        if 'list' in data:
            for forecast in data['list']:
                if "15:00:00" in forecast['dt_txt']:
                    weather_data.append({
                        'day': len(weather_data) + 1,
                        'temperature': forecast['main']['temp'],
                        'humidity': forecast['main']['humidity'],
                        'wind_speed': forecast['wind']['speed'],
                        'precipitation': forecast.get('rain', {}).get('3h', 0)
                    })
        return weather_data
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des données météo: {e}")
        return None

# Ajouter les colonnes météo au DataFrame
for i in range(1, 6):
    df[f'Temperature_Jour_{i}'] = None
    df[f'Humidite_Jour_{i}'] = None
    df[f'Vitesse_Vent_Jour_{i}'] = None
    df[f'Precipitation_Jour_{i}'] = None

for index, row in df.iterrows():
    print(f"Récupération données pour {row['City']}")
    weather_data = get_weather(row['Latitude'], row['Longitude'])
    print(f"Données reçues : {weather_data}")
    if weather_data:
        for day_data in weather_data:
            print(f"Jour {day_data['day']}: {day_data}")
            day = day_data['day']
            df.at[row.name, f'Temperature_Jour_{day}'] = day_data['temperature']
            df.at[row.name, f'Humidite_Jour_{day}'] = day_data['humidity']
            df.at[row.name, f'Vitesse_Vent_Jour_{day}'] = day_data['wind_speed']
            df.at[row.name, f'Precipitation_Jour_{day}'] = day_data['precipitation']

# Enregistrer le DataFrame mis à jour dans un fichier CSV
df.to_csv('src/previsions_meteo.csv', index=False)

# infos sur les données
print(f"\nDonnées mises à jour pour {df['Temperature_Jour_1'].count()} villes sur {len(df)}")

---

## SCRAPPING BOOKING.COM

---

Maintenant que nous avons les données météos des 5 prochains jours pour nos 35 villes. Récupérons une liste d'hotel pour ces villes.

La latitude/longitude nécessitent une requête séparée car :
- Ces données ne sont pas visibles sur la page de recherche principale
- Elles sont uniquement présentes dans le code source de la page détaillée de chaque hôtel
- spécifiquement dans une balise script au format JSON

BeautifulSoup est utilisé car :
- Il permet de cibler spécifiquement les balises script

En résumé : deux étapes de scraping nécessaires pour obtenir toutes les données, avec BeautifulSoup comme outil optionnel mais qui permet une meilleure organisation du code.

In [ ]:
# Fonction pour extraire les coordonnées géographiques d'un hôtel à partir de son URL sur Booking.com
def get_lat_lon_from_booking(url):
    response = requests.get(url) # Effectue une requête HTTP pour obtenir le contenu de la page
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None, None

    soup = BeautifulSoup(response.text, 'html.parser') # Parse le contenu HTML de la réponse

    script_tags = soup.find_all('script') # Trouve tous les éléments <script> dans le document HTML
    for script in script_tags:
        if 'latitude' in script.text and 'longitude' in script.text: # Cherche les scripts contenant des données de latitude et longitude
            lat_lon_match = re.search(r'latitude":([0-9.-]+),"longitude":([0-9.-]+)', script.text) # Extrait les valeurs à l'aide d'une expression régulière
            if lat_lon_match:
                latitude = lat_lon_match.group(1)
                longitude = lat_lon_match.group(2)
                return latitude, longitude

    return None, None

# Définition d'un Spider Scrapy pour l'extraction des données des hôtels sur Booking.com
class BookingSpider(scrapy.Spider):
    name = 'booking'
    allowed_domains = ['booking.com']
    start_urls = [ # URLs pour les recherches initiales sur Booking.com
        'https://www.booking.com/searchresults.html?ss=Mont+Saint+Michel',
        'https://www.booking.com/searchresults.html?ss=St+Malo',
        'https://www.booking.com/searchresults.html?ss=Bayeux',
        'https://www.booking.com/searchresults.html?ss=Le+Havre',
        'https://www.booking.com/searchresults.html?ss=Rouen',
        'https://www.booking.com/searchresults.html?ss=Paris',
        'https://www.booking.com/searchresults.html?ss=Amiens',
        'https://www.booking.com/searchresults.html?ss=Lille',
        'https://www.booking.com/searchresults.html?ss=Strasbourg',
        'https://www.booking.com/searchresults.html?ss=Chateau+du+Haut+Koenigsbourg',
        'https://www.booking.com/searchresults.html?ss=Colmar',
        'https://www.booking.com/searchresults.html?ss=Eguisheim',
        'https://www.booking.com/searchresults.html?ss=Besancon',
        'https://www.booking.com/searchresults.html?ss=Dijon',
        'https://www.booking.com/searchresults.html?ss=Annecy',
        'https://www.booking.com/searchresults.html?ss=Grenoble',
        'https://www.booking.com/searchresults.html?ss=Lyon',
        'https://www.booking.com/searchresults.html?ss=Gorges+du+Verdon',
        'https://www.booking.com/searchresults.html?ss=Bormes+les+Mimosas',
        'https://www.booking.com/searchresults.html?ss=Cassis',
        'https://www.booking.com/searchresults.html?ss=Marseille',
        'https://www.booking.com/searchresults.html?ss=Aix+en+Provence',
        'https://www.booking.com/searchresults.html?ss=Avignon',
        'https://www.booking.com/searchresults.html?ss=Uzes',
        'https://www.booking.com/searchresults.html?ss=Nimes',
        'https://www.booking.com/searchresults.html?ss=Aigues+Mortes',
        'https://www.booking.com/searchresults.html?ss=Saintes+Maries+de+la+mer',
        'https://www.booking.com/searchresults.html?ss=Collioure',
        'https://www.booking.com/searchresults.html?ss=Carcassonne',
        'https://www.booking.com/searchresults.html?ss=Foix',
        'https://www.booking.com/searchresults.html?ss=Toulouse',
        'https://www.booking.com/searchresults.html?ss=Montauban',
        'https://www.booking.com/searchresults.html?ss=Biarritz',
        'https://www.booking.com/searchresults.html?ss=Bayonne',
        'https://www.booking.com/searchresults.html?ss=La+Rochelle'
    ]
    # Configuration pour simuler un navigateur et gérer la fréquence des requêtes
    custom_settings = { 
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
        'AUTOTHROTTLE_ENABLED': True,
        'HTTPCACHE_ENABLED': True
    }
    # Parse les données de chaque hôtel trouvées dans les résultats de recherche
    def parse(self, response):
        city_name = response.url.split('ss=')[-1].replace('+', ' ')

        for hotel in response.xpath("//div[@data-testid='property-card']"):
            hotel_name = hotel.xpath(".//div[@data-testid='title']/text()").get()
            hotel_url = hotel.xpath(".//a[@data-testid='title-link']/@href").get()
            hotel_score = hotel.xpath(".//div[@data-testid='review-score']//div[@class='a3b8729ab1 d86cee9b25']/text()").get()
            hotel_description = hotel.xpath(".//div[@class='abf093bdfe']/text()").get()

            full_hotel_url = response.urljoin(hotel_url) # Construit l'URL complète de la page de l'hôtel

            latitude, longitude = get_lat_lon_from_booking(full_hotel_url)  # Obtient les coordonnées géographiques de l'hôtel

            # Rend les données extraites disponibles pour le traitement ou la sauvegarde
            yield {
                'City': city_name,
                'Hotel_Name': hotel_name,
                'Hotel_URL': full_hotel_url,
                'Score': hotel_score,
                'Description': hotel_description,
                'Latitude': latitude,
                'Longitude': longitude,
            }
# Configuration et démarrage du processus de crawling
process = CrawlerProcess(settings={
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/97.0.4692.99 Safari/537.36',
    'FEEDS': {
        'src/hotels.json': {'format': 'json'},
    },
})

process.crawl(BookingSpider)
process.start()

Ici nous voyons comment nous trouvons les xpatch (capture code source à faire)

Point de compréhesion technique : explication détaillée du code sur le fonctionnement du scraping

1. Importations
```python
import os, re, requests, scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
```
- `os`: Gère les fichiers et chemins
- `re`: Permet d'utiliser les expressions régulières pour chercher des motifs dans du texte
- `requests`: Fait des requêtes web (comme un navigateur)
- `scrapy`: Framework spécialisé pour extraire des données web
- `BeautifulSoup`: Analyse et navigue dans le code HTML

2. Fonction `get_lat_lon_from_booking`
```python
def get_lat_lon_from_booking(url):
```
Cette fonction extrait les coordonnées GPS d'un hôtel sur Booking.com :
- Fait une requête à l'URL de l'hôtel
- Cherche dans le code source les balises `<script>`
- Utilise une expression régulière pour trouver latitude/longitude
- Retourne ces coordonnées ou None si non trouvées

3. Classe `BookingSpider`
```python
class BookingSpider(scrapy.Spider):
```
C'est le "robot" qui va parcourir Booking.com :
- `name = 'booking'`: Identifiant du robot
- `allowed_domains`: Limite le robot à booking.com
- `start_urls`: Liste des pages à visiter (une par ville)
- `custom_settings`: Configuration pour :
  - Simuler un vrai navigateur
  - Contrôler la vitesse des requêtes
  - Mettre en cache les réponses

4. Méthode `parse`
```python
def parse(self, response):
```
Traite chaque page d'hôtel :
- Extrait le nom de la ville de l'URL
- Pour chaque hôtel trouvé :
  - Récupère nom, URL, score, description
  - Obtient les coordonnées GPS
  - Stocke toutes ces informations

5. Configuration finale
```python
process = CrawlerProcess(settings={...})
```
- Configure le processus d'extraction
- Définit le fichier de sortie (hotels.json)
- Démarre le robot

Cette partie du code est donc un "robot" automatisé qui :
1. Visite Booking.com pour chaque ville
2. Récupère les informations de chaque hôtel
3. Enregistre tout dans un fichier JSON

Il utilise des techniques avancées pour :
- Éviter d'être bloqué par Booking.com
- Gérer les erreurs
- Formater proprement les données

In [53]:
# transformation du fichier json en .csv
hotels_df = pd.read_json('src/hotels.json')
hotels_df.to_csv('src/hotels.csv', index=False)
hotels_df.head()

,City,Hotel_Name,Hotel_URL,Score,Description,Latitude,Longitude
0,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,8.0,Le Relais Saint Michel is an hotel facing the ...,48.617587,-1.510396
1,St Malo,Hôtel Anne de Bretagne - Réouverture 2024 aprè...,https://www.booking.com/hotel/fr/anne-de-breta...,9.0,Hôtel Anne de Bretagne - Réouverture 2024 aprè...,48.650641,-2.024754
2,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.en-gb.ht...,8.0,Hotel Vert offers pastel-coloured rooms with a...,48.614700,-1.509617
3,St Malo,"Le Plongeoir, intra-muros",https://www.booking.com/hotel/fr/le-pongeoir-i...,9.1,Managed by a private host,48.648725,-2.027573
4,Bayeux,B&B Nathalie,https://www.booking.com/hotel/fr/imagine-bayeu...,9.5,Managed by a private host,49.276465,-0.706367


---

# Charger les données dans un datalake

---

Nous avons maintenant 2 fichiers CSV contenant des prévisions météos pour les 5 projets jours sur 35 villes.
Et une liste d'hotels par villes.
Transférons ces fichiers dans mon bucket S3 dont j'ai défini le nom dans mon .env. Les crédentials AWS sont elle stocké dans un .secrets

In [4]:

# Charger .env et .secrets pour des configurations générales
load_dotenv(".env")
load_dotenv(".secrets")

# Accéder aux variables d'environnement
BUCKET_NAME = os.getenv("S3_BUCKET")
AWS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET = os.getenv("AWS_SECRET_ACCESS_KEY")

# Vérification des variables d'environnement
if not BUCKET_NAME or not AWS_KEY or not AWS_SECRET:
    raise ValueError("One or more environment variables are missing")

# Création du client S3
s3 = boto3.client("s3", 
                  aws_access_key_id=AWS_KEY, 
                  aws_secret_access_key=AWS_SECRET)

# transférer les fichiers dans le bucket
s3.upload_file("src/hotels.csv", BUCKET_NAME, "kayak_project/hotels.csv")
s3.upload_file("src/previsions_meteo.csv", BUCKET_NAME, "kayak_project/previsions_meteo.csv")



---

## DATA WHAREHOUSE ET REQUETES SQL

---

Puisqu'il n'y a pas beaucoup de données, j'ai fait le choix de travailler sous NEONDB (postgresql serverless). Cela permet d'avoir une solution gratuite et pédagogique.
L'url de la base postgres contenant les id et pass, est intégré dans le .secret
Il faut maintenant créer les 2 tables "hotels" et "previsions_meteo" avant de pouvoir y transférer les infos de mes .CSV

In [5]:
# Charger les variables d'environnement
load_dotenv('.secrets')

# Récupérer l'URL de la base de données depuis les variables d'environnement
DATABASE_URL = os.getenv('NEON_DATABASE_URL')

# Créer le moteur de connexion
engine = create_engine(DATABASE_URL)

# Définition des requêtes SQL de création des tables
create_hotels_table = """
CREATE TABLE IF NOT EXISTS hotels (
    id SERIAL PRIMARY KEY,
    "City" VARCHAR(100),
    "Hotel_Name" VARCHAR(200),
    "Hotel_URL" TEXT,
    "Score" DECIMAL(3,1),
    "Description" TEXT,
    "Latitude" DECIMAL(10,8),
    "Longitude" DECIMAL(11,8)
);
"""

create_meteo_table = """
CREATE TABLE IF NOT EXISTS previsions_meteo (
    id SERIAL PRIMARY KEY,
    "City" VARCHAR(100),
    "Latitude" DECIMAL(10,8),
    "Longitude" DECIMAL(11,8),
    "Temperature_Jour_1" DECIMAL(4,1),
    "Humidite_Jour_1" INTEGER,
    "Vitesse_Vent_Jour_1" DECIMAL(5,2),
    "Precipitation_Jour_1" DECIMAL(5,2),
    "Temperature_Jour_2" DECIMAL(4,1),
    "Humidite_Jour_2" INTEGER,
    "Vitesse_Vent_Jour_2" DECIMAL(5,2),
    "Precipitation_Jour_2" DECIMAL(5,2),
    "Temperature_Jour_3" DECIMAL(4,1),
    "Humidite_Jour_3" INTEGER,
    "Vitesse_Vent_Jour_3" DECIMAL(5,2),
    "Precipitation_Jour_3" DECIMAL(5,2),
    "Temperature_Jour_4" DECIMAL(4,1),
    "Humidite_Jour_4" INTEGER,
    "Vitesse_Vent_Jour_4" DECIMAL(5,2),
    "Precipitation_Jour_4" DECIMAL(5,2),
    "Temperature_Jour_5" DECIMAL(4,1),
    "Humidite_Jour_5" INTEGER,
    "Vitesse_Vent_Jour_5" DECIMAL(5,2),
    "Precipitation_Jour_5" DECIMAL(5,2)
);
"""

# Exécution des requêtes
with engine.connect() as conn:
    conn.execute(text(create_hotels_table))
    conn.execute(text(create_meteo_table))
    conn.commit()

print("Les tables ont été créées avec succès!")

Les tables ont été créées avec succès!


Récupérons la dernière version des fichiers pour mettre à jour les 2 tables :

In [56]:
# récupérer les fichiers sur le bucket
s3.download_file(BUCKET_NAME, "kayak_project/hotels.csv", "src/hotels.csv")
s3.download_file(BUCKET_NAME, "kayak_project/previsions_meteo.csv", "src/previsions_meteo.csv")


Finissons par transférer les infos des .csv dans nos deux tables :

In [2]:
# Charger les variables d'environnement depuis .secrets
load_dotenv('.secrets')

# Récupérer l'URL de la base de données
DATABASE_URL = os.getenv('NEON_DATABASE_URL')
engine = create_engine(DATABASE_URL)

try:
    # Lire les fichiers CSV locaux
    hotels_df = pd.read_csv('src/hotels.csv')
    meteo_df = pd.read_csv('src/previsions_meteo.csv')

    # Créer une nouvelle connexion pour les opérations to_sql
    with engine.connect() as conn:
        # Démarrer une nouvelle transaction
        with conn.begin():
            # Charger les données dans la base de données
            hotels_df.to_sql('hotels', conn, if_exists='replace', index=False)
            meteo_df.to_sql('previsions_meteo', conn, if_exists='replace', index=False)
        
    print("Les données ont été chargées avec succès!")

except Exception as e:
    print(f"Une erreur s'est produite : {e}")
    # La transaction sera automatiquement annulée en cas d'erreur

finally:
    # Fermer proprement la connexion
    engine.dispose()

# Vérifier le nombre de lignes chargées
with engine.connect() as conn:
    result_hotels = conn.execute(text("SELECT COUNT(*) FROM hotels")).fetchone()
    result_meteo = conn.execute(text("SELECT COUNT(*) FROM previsions_meteo")).fetchone()
    
print(f"Nombre d'hôtels chargés : {result_hotels[0]}")
print(f"Nombre de prévisions météo chargées : {result_meteo[0]}")

Les données ont été chargées avec succès!
Nombre d'hôtels chargés : 876
Nombre de prévisions météo chargées : 35


Avant la phase de visualisation, une petite requête :

In [3]:
# Requête SQL pour récupérer les 3 hotels les mieux notés dans les gorges du Verdon
query = """
SELECT "Hotel_Name", "Score"
FROM hotels
WHERE "City" = 'Gorges du Verdon'
ORDER BY "Score" DESC
LIMIT 3
"""

# Exécuter la requête
with engine.connect() as conn:
    result = pd.read_sql_query(query, conn)

# Afficher les résultats
print(result)

                 Hotel_Name  Score
0         Les Chênes Blancs   10.0
1                Coquelicot    9.7
2  Maison de village type 2    9.5


---

## visualisation ##

---

On nous demande de pouvoir visualisser les 5 meilleurs destinations. Puis les 20 hotels de ces 5 destinations.
J'ai créé plusieurs critères météos dans l'idée de pouvoir laisser le choix à l'utilisateur final en fonction de la demande dui client. Mais dans un premier temps je vais me concentrer sur une carte simple "des 5 destinations avec la moyenne de température la plus chaudes prévu dans les 5 prochains jours".

In [12]:
# requêtes simples pour visualiser juste les villes

# initier les dataframes
hotels_df = pd.read_csv('src/hotels.csv')
meteo_df = pd.read_csv('src/previsions_meteo.csv')

# Les 5 villes les moins venteuses (basé sur la moyenne des vents à 15h sur 5 jours)
query_meteo_vent = """
SELECT 
   "City",
   ("Vitesse_Vent_Jour_1" + "Vitesse_Vent_Jour_2" + "Vitesse_Vent_Jour_3" + 
    "Vitesse_Vent_Jour_4" + "Vitesse_Vent_Jour_5")/5 as vitesse_moyenne_vent,
   "Latitude", "Longitude"
FROM previsions_meteo
ORDER BY vitesse_moyenne_vent ASC
LIMIT 5;
"""

# Les 5 villes les plus chaudes (basé sur la moyenne des températures)
query_meteo_chaud = """
SELECT 
   "City",
   ("Temperature_Jour_1" + "Temperature_Jour_2" + "Temperature_Jour_3" + 
    "Temperature_Jour_4" + "Temperature_Jour_5")/5 as temperature_moyenne,
   "Latitude", "Longitude"
FROM previsions_meteo
ORDER BY temperature_moyenne DESC
LIMIT 5;
"""

# Les 5 villes les plus froides (basé sur la moyenne des températures
query_meteo_froid = """
SELECT 
   "City",
   ("Temperature_Jour_1" + "Temperature_Jour_2" + "Temperature_Jour_3" + 
    "Temperature_Jour_4" + "Temperature_Jour_5")/5 as temperature_moyenne,
   "Latitude", "Longitude"
FROM previsions_meteo
ORDER BY temperature_moyenne ASC
LIMIT 5;
"""

# Les 5 villes ou il pleuvra le moins (basé sur la somme des précipitations ensigés sur 5 jours)
query_meteo_sec = """
SELECT 
   "City",
   ("Precipitation_Jour_1" + "Precipitation_Jour_2" + "Precipitation_Jour_3" + 
    "Precipitation_Jour_4" + "Precipitation_Jour_5") as precipitation_totale,
   "Latitude", "Longitude"
FROM previsions_meteo
ORDER BY precipitation_totale ASC
LIMIT 5;
"""

# Exécution des requêtes
with engine.connect() as conn:
    villes_vent = pd.read_sql_query(query_meteo_vent, conn)
    villes_chaudes = pd.read_sql_query(query_meteo_chaud, conn)
    villes_froides = pd.read_sql_query(query_meteo_froid, conn)
    villes_seches = pd.read_sql_query(query_meteo_sec, conn)

print("Villes les moins venteuses:")
print(villes_vent)
print("Villes les plus chaudes:")
print(villes_chaudes)
print("Villes les plus froides:")
print(villes_froides)
print("Villes les moins pluvieuses:")
print(villes_seches)

Villes les moins venteuses:
                           City  vitesse_moyenne_vent   Latitude  Longitude
0                          Foix                 0.796  42.963900   1.605381
1  Chateau du Haut Koenigsbourg                 0.902  48.249523   7.345492
2               Aix en Provence                 0.976  43.529842   5.447474
3                        Colmar                 1.256  48.077752   7.357964
4                     Eguisheim                 1.308  48.044797   7.307962
Villes les plus chaudes:
                City  temperature_moyenne   Latitude  Longitude
0            Bayonne               12.562  43.494514  -1.473666
1           Biarritz               12.476  43.471144  -1.552727
2          Marseille               11.872  43.296174   5.369953
3          Collioure               11.796  42.525050   3.083155
4  Mont Saint Michel               11.440  48.635954  -1.511460
Villes les plus froides:
                           City  temperature_moyenne   Latitude  Longitude
0  Chat

In [13]:
# visualisation simple
fig = px.scatter_mapbox(villes_chaudes, lat="Latitude", lon="Longitude", hover_name="City", 
                        hover_data=["temperature_moyenne"],
                        color="temperature_moyenne", color_continuous_scale=["blue", "purple", "red"], 
                        zoom=4, height=300)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Travaillons la même carte en rajoutant les 20 meilleurs hotels par ville.

### Explication de la requête SQL

La requête SQL suivante est utilisée pour obtenir les 20 meilleurs hôtels dans les 5 villes les moins venteuses. Elle utilise deux tables : `previsions_meteo` et `hotels`.

La clause WITH en SQL, également connue sous le nom de Common Table Expression (CTE), permet de définir des sous-requêtes temporaires qui peuvent être référencées dans la requête principale. Cela ne signifie pas nécessairement que les données proviennent de deux tables, mais plutôt que la requête utilise des sous-requêtes pour structurer et organiser les données avant de les utiliser dans la requête principale.

Ici, la requête utilise deux CTEs pour structurer les données avant de les sélectionner dans la requête principale.

### Structure de la requête

La requête est structurée en deux parties principales, chacune définie dans une clause WITH (Common Table Expression, CTE), suivie d'une sélection finale :

VillesMoinsVenteuses : Cette CTE sélectionne les 5 villes avec la vitesse moyenne du vent la plus basse.
TopHotels : Cette CTE sélectionne les meilleurs hôtels dans les villes identifiées par VillesMoinsVenteuses.

L'alias h est utilisé pour référencer la table hotels dans la sous-requête TopHotels.
L'alias v est utilisé pour référencer la CTE VillesMoinsVenteuses dans la sous-requête TopHotels.



In [ ]:
# Requêtes SQL pour obtenir les 20 hôtels les mieux noté sur les 5 villes les plus ou les moins "faire son choix"
query_hotels_vent = """
WITH VillesMoinsVenteuses AS (
    SELECT "City", 
           ("Vitesse_Vent_Jour_1" + "Vitesse_Vent_Jour_2" + "Vitesse_Vent_Jour_3" + "Vitesse_Vent_Jour_4" + "Vitesse_Vent_Jour_5")/5 as vitesse_moyenne_vent,
           "Latitude", "Longitude"
    FROM previsions_meteo
    ORDER BY vitesse_moyenne_vent ASC
    LIMIT 5
),
TopHotels AS (
    SELECT 
        v."City",
        v.vitesse_moyenne_vent,
        h."Hotel_Name",
        h."Score",
        h."Hotel_URL",
        h."Latitude",
        h."Longitude",
        ROW_NUMBER() OVER (PARTITION BY v."City" ORDER BY h."Score" DESC) as rang
    FROM VillesMoinsVenteuses v
    JOIN hotels h ON v."City" = h."City"
)
SELECT 
    "City",
    vitesse_moyenne_vent,
    "Hotel_Name",
    "Score",
    "Hotel_URL",
    "Latitude",
    "Longitude"
FROM TopHotels
WHERE rang <= 20
ORDER BY vitesse_moyenne_vent ASC, "Score" DESC;
"""

query_hotels_chaud = """
WITH VillesChaudes AS (
    SELECT 
        "City",
        ("Temperature_Jour_1" + "Temperature_Jour_2" + "Temperature_Jour_3" + 
         "Temperature_Jour_4" + "Temperature_Jour_5")/5 as temperature_moyenne,
        "Latitude", "Longitude"
    FROM previsions_meteo
    ORDER BY temperature_moyenne DESC
    LIMIT 5
), 
TopHotels AS (
    SELECT 
        v."City",
        v.temperature_moyenne,
        h."Hotel_Name",
        h."Score",
        h."Hotel_URL",
        h."Latitude",
        h."Longitude",
        ROW_NUMBER() OVER (PARTITION BY v."City" ORDER BY h."Score" DESC) as rang
    FROM VillesChaudes v
    JOIN hotels h ON v."City" = h."City"
)
SELECT 
    "City",
    temperature_moyenne,
    "Hotel_Name",
    "Score",
    "Hotel_URL",
    "Latitude",
    "Longitude"
FROM TopHotels 
WHERE rang <= 20 
ORDER BY temperature_moyenne DESC, "Score" DESC;
"""

query_hotels_froid = """
WITH VillesFroides AS (
    SELECT 
        "City",
        ("Temperature_Jour_1" + "Temperature_Jour_2" + "Temperature_Jour_3" + 
         "Temperature_Jour_4" + "Temperature_Jour_5")/5 as temperature_moyenne,
        "Latitude", "Longitude"
    FROM previsions_meteo
    ORDER BY temperature_moyenne ASC
    LIMIT 5
), 
TopHotels AS (
    SELECT 
        v."City",
        v.temperature_moyenne,
        h."Hotel_Name",
        h."Score",
        h."Hotel_URL",
        h."Latitude",
        h."Longitude",
        ROW_NUMBER() OVER (PARTITION BY v."City" ORDER BY h."Score" DESC) as rang
    FROM Villesfroides v
    JOIN hotels h ON v."City" = h."City"
)
SELECT 
    "City",
    temperature_moyenne,
    "Hotel_Name",
    "Score",
    "Hotel_URL",
    "Latitude",
    "Longitude"
FROM TopHotels 
WHERE rang <= 20 
ORDER BY temperature_moyenne DESC, "Score" ASC;
"""

query_hotels_sec = """
WITH VillesSeches AS (
    SELECT "City",
           ("Precipitation_Jour_1" + "Precipitation_Jour_2" + "Precipitation_Jour_3" + "Precipitation_Jour_4" + "Precipitation_Jour_5") as precipitation_totale,
           "Latitude", "Longitude"
    FROM previsions_meteo
    ORDER BY precipitation_totale ASC
    LIMIT 5
),
TopHotels AS (
    SELECT 
        v."City",
        v.precipitation_totale,
        h."Hotel_Name",
        h."Score",
        h."Hotel_URL",
        h."Latitude",
        h."Longitude",
        ROW_NUMBER() OVER (PARTITION BY v."City" ORDER BY h."Score" DESC) as rang
    FROM VillesSeches v
    JOIN hotels h ON v."City" = h."City"
)
SELECT 
    "City",
    precipitation_totale,
    "Hotel_Name",
    "Score",
    "Hotel_URL",
    "Latitude",
    "Longitude"
FROM TopHotels
WHERE rang <= 20
ORDER BY precipitation_totale ASC, "Score" DESC;
"""

# Exécution des requêtes
with engine.connect() as conn:
    hotels_villes_calmes = pd.read_sql_query(query_hotels_vent, conn)
    hotels_villes_chaudes = pd.read_sql_query(query_hotels_chaud, conn)
    hotels_villes_froides = pd.read_sql_query(query_hotels_froid, conn)
    hotels_villes_seches = pd.read_sql_query(query_hotels_sec, conn)

print("Hôtels dans les villes les moins venteuses:")
print(hotels_villes_calmes)
print("Hôtels dans les villes les plus chaudes:")
print(hotels_villes_chaudes)
print("Hôtels dans les villes les plus froides:")
print(hotels_villes_froides)
print("Hôtels dans les villes les moins pluvieuses:")
print(hotels_villes_seches)

Hôtels dans les villes les moins venteuses:
         City  vitesse_moyenne_vent  \
0        Foix                 0.796   
1        Foix                 0.796   
2        Foix                 0.796   
3        Foix                 0.796   
4        Foix                 0.796   
..        ...                   ...   
95  Eguisheim                 1.308   
96  Eguisheim                 1.308   
97  Eguisheim                 1.308   
98  Eguisheim                 1.308   
99  Eguisheim                 1.308   

                                           Hotel_Name  Score  \
0   cocoon 52 m2 new, beautiful view castle and mo...    9.3   
1   Studio Le Flore - Petit déjeuner inclus 1ère n...    9.3   
2                                   Un second souffle    9.1   
3                              l'Arche des Chapeliers    9.1   
4   Lit King Size Trés Cosy Hyper Centre de Foix C...    9.0   
..                                                ...    ...   
95                      Hotel Colmar Vi

Terminons par affichier les cartes comme demandés dans le projet. Pour la deuxième carte, faites un zoom sur la zone de Biarritz

In [16]:
# Visualisation des 5 villes les plus chaudes
fig = px.scatter_mapbox(villes_chaudes, lat="Latitude", lon="Longitude", hover_name="City", 
                        hover_data=["temperature_moyenne"],
                        color="temperature_moyenne", color_continuous_scale=["blue", "purple", "red"], 
                        zoom=4, height=300)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(title="Les 5 villes les plus chaudes en France")
fig.show()

# Visualisation des 20 meilleurs hôtels dans les villes les plus chaudes
fig = px.scatter_mapbox(hotels_villes_chaudes, lat="Latitude", lon="Longitude", hover_name="Hotel_Name",
                        hover_data=["Score"],
                        color="Score", color_continuous_scale="Viridis", 
                        zoom=4, height=300)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(title="Les 20 meilleurs hôtels dans les villes les plus chaudes")
fig.show()

### Amélioration de la visualisation
Maintenant il serait intéressant d'avoir une carte interactive ou il est possible de choisir ses critères météos et de pouvoir également cliquer sur les hotels.

In [ ]:
# travail optionnel : test création carte interactive avec folium et hebergement sur github.io

def create_weather_map(resultats):
    # Créer une carte centrée sur la France
    m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)
    
    # Créer les groupes principaux pour chaque type de météo
    layers = {}
    
    # Définir des couleurs avec plus de contraste
    color_scale = {
        1: '#FF0000',    # Rouge vif
        2: '#9400D3',    # Violet profond
        3: '#0000FF',    # Bleu pur
        4: '#006400',    # Vert foncé
        5: '#000080'     # Bleu marine
    }
    
    # Opacités différentes pour distinguer ville principale et hôtels
    CITY_OPACITY = 0.9
    HOTEL_OPACITY = 0.6
    
    for weather_type in resultats.keys():
        layers[weather_type] = folium.FeatureGroup(name=f"Villes {weather_type}")
    
    for weather_type, df in resultats.items():
        # Grouper par ville
        cities = df.groupby('City').agg({
            'Latitude': 'first',
            'Longitude': 'first',
            'Score': ['mean', 'count']
        }).reset_index()
        
        cities.columns = ['City', 'Latitude', 'Longitude', 'Score_mean', 'Hotel_count']
        cities = cities.sort_values('Score_mean', ascending=False)
        
        # Ajouter les marqueurs des villes
        for idx, city in cities.iterrows():
            rank = idx + 1
            color = color_scale[min(rank, 5)]
            
            # Liste des hôtels pour le popup
            hotels_list = df[df['City'] == city['City']].sort_values('Score', ascending=False)
            hotels_html = "".join([
                f"""
                <tr>
                    <td style='padding:2px'>{hotel['Hotel_Name']}</td>
                    <td style='padding:2px;text-align:right'>{hotel['Score']:.1f}</td>
                </tr>
                """ for _, hotel in hotels_list.iterrows()
            ])
            
            # Popup pour la ville avec style amélioré
            city_popup = folium.Popup(
                f"""
                <div style='width:300px'>
                <h4 style='margin:0;padding:5px;background-color:{color};color:white'>
                    {city['City']} (Rang: {rank})
                </h4>
                <div style='padding:5px'>
                    <b>Score moyen:</b> {city['Score_mean']:.1f}<br>
                    <b>Nombre d'hôtels:</b> {city['Hotel_count']}
                </div>
                <div style='max-height:200px;overflow:auto'>
                    <table style='width:100%;border-collapse:collapse'>
                        <tr style='background-color:#f0f0f0'>
                            <th style='text-align:left;padding:5px'>Hôtel</th>
                            <th style='text-align:right;padding:5px'>Score</th>
                        </tr>
                        {hotels_html}
                    </table>
                </div>
                </div>
                """,
                max_width=300
            )
            
            # Marqueur principal pour la ville
            folium.CircleMarker(
                location=[city['Latitude'], city['Longitude']],
                radius=7,
                popup=city_popup,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=CITY_OPACITY,
                weight=2
            ).add_to(layers[weather_type])
            
            # Hôtels autour
            for _, hotel in hotels_list.iterrows():
                lat_offset = random.uniform(-0.001, 0.001)
                lon_offset = random.uniform(-0.001, 0.001)
                
                folium.CircleMarker(
                    location=[
                        hotel['Latitude'] + lat_offset,
                        hotel['Longitude'] + lon_offset
                    ],
                    radius=3,
                    popup=folium.Popup(
                        f"""
                        <div style='width:200px'>
                        <h4 style='margin:0;padding:5px;background-color:{color};color:white'>
                            {hotel['Hotel_Name']}
                        </h4>
                        <div style='padding:5px'>
                            Score: {hotel['Score']:.1f}<br>
                            <a href="{hotel['Hotel_URL']}" target="_blank">Voir sur Booking</a>
                        </div>
                        </div>
                        """,
                        max_width=300
                    ),
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=HOTEL_OPACITY,
                    weight=1
                ).add_to(layers[weather_type])
    
    # Ajouter les couches à la carte
    for layer in layers.values():
        layer.add_to(m)
    
    # Contrôle des couches
    folium.LayerControl(collapsed=False).add_to(m)
    
    # Légende avec les nouvelles couleurs
    legend_html = """
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 150px;
                border:2px solid grey; z-index:9999; 
                background-color:white;
                padding: 10px;
                font-size: 14px;
                opacity: 0.9;
                ">
    <b>Classement</b><br>
    <i class="fa fa-circle" style="color:#FF0000"></i> 1er<br>
    <i class="fa fa-circle" style="color:#9400D3"></i> 2ème<br>
    <i class="fa fa-circle" style="color:#0000FF"></i> 3ème<br>
    <i class="fa fa-circle" style="color:#006400"></i> 4ème<br>
    <i class="fa fa-circle" style="color:#000080"></i> 5ème
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Utilisation :
try:
    # Créer la carte avec nos résultats
    carte = create_weather_map(resultats)
    
    # Sauvegarder la carte en HTML
    carte.save('carte_hotels.html')
    print("Carte créée avec succès ! Ouvrez carte_hotels.html dans votre navigateur.")
except Exception as e:
    print(f"Erreur lors de la création de la carte : {e}")

Carte créée avec succès ! Ouvrez carte_hotels.html dans votre navigateur.


La carte est assez fonctionnelle mais mérites des retouches pour améliorer l'expérience utilisateur.